In [12]:
import pygame
from pygame.locals import *
import pygame.sprite
import random

In [13]:
seaColor = (12, 15, 95)
hitColor = (211, 0, 0)
missColor = (230, 230, 230)
width = 586
height = 427

In [14]:
class Sprite(pygame.sprite.Sprite):
    def __init__(self, width, height):
        super().__init__()
        
        self.image = pygame.Surface((width, height))
        self.image.fill(seaColor)
        self.rect = self.image.get_rect()
        self.empty = False
        
        

In [15]:
screen = pygame.display.set_mode((width, height))
clock = pygame.time.Clock()

In [16]:
sprites = pygame.sprite.Group()

In [17]:
spacing = 3
totSpacing = spacing * 11
spriteSize = 30

In [18]:
sprite_positions = [
    ((spriteSize + spacing) * col + spacing, (spriteSize + spacing) * row + spacing)
    for row in range(10)
    for col in range(10)
]

In [19]:
for position in sprite_positions:
    sprite = Sprite(spriteSize, spriteSize)
    sprite.image = pygame.Surface((spriteSize, spriteSize))
    sprite.image.fill(seaColor) 
    sprite.rect = sprite.image.get_rect()
    sprite.rect.topleft = position
    sprites.add(sprite)

In [20]:
sizes = [5, 4, 3, 3, 2]
valid = True
for size in sizes:  #for each size ship
    found = False
    while not found:    # while ship position is not found
        randomNum = random.randint(0, 99)
        if (randomNum + (size * 10) <= 99):     # if ship position is within grid size
            j = randomNum
            while (j < randomNum + (size * 10)):
                if (sprites.sprites()[j].empty == True):
                    valid = False
                j += 10
            if valid:
                j = randomNum
                while (j < randomNum + (size * 10)):
                    sprites.sprites()[j].empty = True
                    j += 10    
                found = True
            

a
a
a
a
a
b
a
a
a
a
b
a
a
a
b
a
a
a
b
a
a
b


In [21]:
def main():
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONUP:
                pos = pygame.mouse.get_pos()
                clicked_sprites = [sprite for sprite in sprites if sprite.rect.collidepoint(pos)]
                for sprite in clicked_sprites:
                    sprite_width, sprite_height = sprite.image.get_size()
                    local_pos = pos[0] - sprite.rect.left, pos[1] - sprite.rect.top
                    if (0 <= local_pos[0] < sprite_width) and (0 <= local_pos[1] < sprite_height):
                        if (sprite.image.get_at(local_pos) == seaColor and sprite.empty) :
                            sprite.image.fill(hitColor)
                        else:
                            sprite.image.fill(missColor)

        sprites.update()
        
        screen.fill((0, 0, 0))
        sprites.draw(screen)

        pygame.display.flip()
        clock.tick(60)

In [22]:
if __name__ == "__main__":
    pygame.init()
    main()
    
pygame.quit()